In [ ]:
import os
import pickle

directory = "results/"

exp_results = {}
abl_linear = {}
abl_kan = {}

for filename in os.listdir(directory):
    if filename.endswith(".pickle") and not filename.startswith("ablation_"):
        with open(os.path.join(directory, filename), "rb") as f:
            data = pickle.load(f)
        exp_results[filename.replace(".pickle", "")] = data

    if filename.startswith("ablation_") and filename.find("kan")==-1:
        with open(os.path.join(directory, filename), "rb") as f:
            data = pickle.load(f)
        abl_linear[filename.replace(".pickle", "")] = data

    if filename.startswith("ablation_") and filename.find("kan")!=-1:
        with open(os.path.join(directory, filename), "rb") as f:
            data = pickle.load(f)
        abl_kan[filename.replace(".pickle", "")] = data

exp_results = dict(sorted(exp_results.items()))
abl_linear = dict(sorted(abl_linear.items()))
abl_kan = dict(sorted(abl_kan.items()))

print(len(exp_results.keys()), len(abl_linear.keys()), len(abl_kan.keys()))
print(exp_results.keys(),"\n",abl_linear.keys(),"\n",abl_kan.keys())

In [ ]:
overall_metrics = [
    "Overall ACC",
    "F1 Macro",
    "F1 Micro",
    "Kappa",
    # "TPR Micro",
    # "TPR Macro",
    # "TNR Micro",
    # "TNR Macro",
    # "FPR Macro",
    # "FNR Macro",
    # "PPV Macro",
    # "NPV Macro",
    # "FPR Micro",
    # "FNR Micro",
    # "PPV Micro",
    # "NPV Micro",
]

In [ ]:
import pandas as pd

all_overall_metrics = []

for model in exp_results.keys():
    all_overall_metrics.append(
        pd.DataFrame(
            {
                k: exp_results[model]["test_metrics"].overall_stat[k]
                for k in overall_metrics
                if k in exp_results[model]["test_metrics"].overall_stat
            },
            index=[[model]],
        )
    )

In [ ]:
overall_results = (
    pd.concat(all_overall_metrics).reset_index().rename(columns={"level_0":"experiment"})
)

In [ ]:
overall_results[["dataset", "model", "classifier"]] = (
    overall_results.experiment.str.split("_", expand=True)
)

overall_results = overall_results.drop(columns=["experiment"])
overall_results = overall_results.set_index(["dataset", "model", "classifier"])

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

overall_results

In [ ]:
best_models

In [ ]:
epoch_results_kan = []

for k, v in best_models.items():
    for i in range(1,6):
        epoch_results_kan.append(
            {
                f"{k}_{v}_{'kan'}": [
                    i,
                    exp_results[f"{k}_{v}_{'kan'}"][f"epoch_{i}"]["training_loss"],
                    exp_results[f"{k}_{v}_{'kan'}"][f"epoch_{i}"]["validation_loss"],
                    exp_results[f"{k}_{v}_{'kan'}"][f"epoch_{i}"][
                        "validation_metrics"
                    ].overall_stat["Overall ACC"],
                    exp_results[f"{k}_{v}_{'kan'}"][f"epoch_{i}"][
                        "validation_metrics"
                    ].overall_stat["F1 Macro"],
                    exp_results[f"{k}_{v}_{'kan'}"][f"epoch_{i}"][
                        "validation_metrics"
                    ].overall_stat["Kappa"],
                ]
            }
        )

pd.concat(
    [
        (
            pd.DataFrame(
                result,
                index=[
                    "epoch",
                    "training_loss",
                    "validation_loss",
                    "Overall ACC",
                    "F1 Macro",
                    "Kappa",
                ],
            ).T
        )
        for result in epoch_results_kan
    ]
)

In [ ]:
epoch_results_mlp = []

for k, v in best_models.items():
    for i in range(1, 6):
        epoch_results_mlp.append(
            {
                f"{k}_{v}_{'linear'}": [
                    i,
                    exp_results[f"{k}_{v}_{'linear'}"][f"epoch_{i}"]["training_loss"],
                    exp_results[f"{k}_{v}_{'linear'}"][f"epoch_{i}"]["validation_loss"],
                    exp_results[f"{k}_{v}_{'linear'}"][f"epoch_{i}"][
                        "validation_metrics"
                    ].overall_stat["Overall ACC"],
                    exp_results[f"{k}_{v}_{'linear'}"][f"epoch_{i}"][
                        "validation_metrics"
                    ].overall_stat["F1 Macro"],
                    exp_results[f"{k}_{v}_{'linear'}"][f"epoch_{i}"][
                        "validation_metrics"
                    ].overall_stat["Kappa"],
                ]
            }
        )

pd.concat(
    [
        (
            pd.DataFrame(
                result,
                index=[
                    "epoch",
                    "training_loss",
                    "validation_loss",
                    "Overall ACC",
                    "F1 Macro",
                    "Kappa",
                ],
            ).T
        )
        for result in epoch_results_mlp
    ]
)

In [ ]:
best_models = {
    "agnews": "distilbert",
    "dbpedia": "distilbert",
    "imdb": "distilbert",
    "papluca": "distilbert",
    "sst5": "bert",
    "trec": "bart",
    "yelp": "distilbert",
}

In [ ]:
abl_overall_linear_metrics = []

for model in abl_linear.keys():
    abl_overall_linear_metrics.append(
        pd.DataFrame(
            {
                k: abl_linear[model]["test_metrics"].overall_stat[k]
                for k in overall_metrics
                if k in abl_linear[model]["test_metrics"].overall_stat
            },
            index=[[model]],
        )
    )

abl_overall_linear_results = (
    pd.concat(abl_overall_linear_metrics)
    .reset_index()
    .rename(columns={"level_0": "experiment"})
)

abl_overall_linear_results.experiment = (
    abl_overall_linear_results.experiment.str.replace("ablation_", "")
)

abl_overall_linear_results[["dataset", "model", "hidden_neurons_fc1", "trainable_parameters"]] = (
    abl_overall_linear_results.experiment.str.split("_", expand=True)
)

abl_overall_linear_results = abl_overall_linear_results.drop(columns=["experiment"])
abl_overall_linear_results["classifier"] = "mlp"
abl_overall_linear_results = abl_overall_linear_results.set_index(
    ["dataset", "model", "classifier"]
)

abl_overall_linear_results

In [ ]:
kan_params = {
    ("agnews", "distilbert", "kan"): 30724,
    ("dbpedia", "distilbert", "kan"): 107534,
    ("imdb", "distilbert", "kan"): 15362,
    ("papluca", "distilbert", "kan"): 153620,
    ("sst5", "bert", "kan"): 38405,
    ("trec", "bart", "kan"): 512050,
    ("yelp", "distilbert", "kan"): 38405,
}

pd.concat(
    [
        overall_results.loc[
            [(d, m, "kan") for d, m in list(abl_overall_linear_results.index)]
        ],
        pd.DataFrame(
            kan_params.values(),
            index=kan_params.keys(),
            columns=["trainable_parameters"],
        ).assign(grid_size=5),
    ],
    axis=1,
).rename_axis(("dataset", "model", "classifier"))

In [ ]:
# overall_results.loc[[(d, m, "linear") for d, m in list(abl_overall_linear_results.index)]]

In [ ]:
abl_overall_kan_metrics = []

for model in abl_kan.keys():
    abl_overall_kan_metrics.append(
        pd.DataFrame(
            {
                k: abl_kan[model]["test_metrics"].overall_stat[k]
                for k in overall_metrics
                if k in abl_kan[model]["test_metrics"].overall_stat
            },
            index=[[model]],
        )
    )

abl_overall_kan_results = (
    pd.concat(abl_overall_kan_metrics)
    .reset_index()
    .rename(columns={"level_0": "experiment"})
)

abl_overall_kan_results.experiment = abl_overall_kan_results.experiment.str.replace("ablation_", "")
abl_overall_kan_results.experiment = abl_overall_kan_results.experiment.str.replace("kan-", "")

abl_overall_kan_results[["dataset", "model", "grid_size"]] = abl_overall_kan_results.experiment.str.split("_", expand=True)
abl_overall_kan_results.grid_size = abl_overall_kan_results.grid_size.astype(int)
abl_overall_kan_results = abl_overall_kan_results.drop(columns=["experiment"])
abl_overall_kan_results = abl_overall_kan_results[
    ["dataset", "model", "grid_size", "Overall ACC", "F1 Macro", "F1 Micro", "Kappa"]
]

abl_overall_kan_results = pd.concat(
    [
        abl_overall_kan_results,
        overall_results.loc[
            [(d, m, "kan") for d, m in list(abl_overall_linear_results.index)]
        ]
        .reset_index()
        .rename(columns={"classifier": "grid_size"})
        .assign(grid_size=5)
    ]
).reset_index(drop=True)

abl_overall_kan_results = abl_overall_kan_results.sort_values(by=["dataset", "model", "grid_size"])
abl_overall_kan_results = abl_overall_kan_results.set_index(["dataset", "model"])

abl_overall_kan_results

In [ ]:
data_desc = {
    "ag_news": ["News Media", 15000, 10500, 2250, 2250, 4],
    "dbpedia_14": ["Ontology", 15000, 10500, 2250, 2250, 14],
    "imdb": ["Review", 15000, 10500, 2250, 2250, 2],
    "papluca": ["Language Detection", 15000, 10500, 2250, 2250, 20],
    "sst5": ["Sentiment", 11855, 8298, 1778, 1779, 5],
    "trec": ["Question/Answer", 5952, 4166, 893, 893, 50],
    "yelp": ["Review", 15000, 10500, 2250, 2250, 5],
}

pd.DataFrame(
    data_desc.values(),
    index=data_desc.keys(),
    columns=[
        "Domain",
        "Sample Size",
        "Training Size",
        "Validation Size",
        "Test Size",
        "Number of Labels",
    ],
)

In [ ]:
with open("results/imdb_distilbert_kan--9.pickle", "rb") as f:
    imdb_distilbert_kan_9 = pickle.load(f)

with open("results/imdb_distilbert_kan--7.pickle", "rb") as f:
    imdb_distilbert_kan_7 = pickle.load(f)
    
pd.DataFrame(
    [
        [   7,
            imdb_distilbert_kan_7["test_metrics"].overall_stat["Overall ACC"],
            imdb_distilbert_kan_7["test_metrics"].overall_stat["F1 Macro"],
            imdb_distilbert_kan_7["test_metrics"].overall_stat["F1 Micro"],
            imdb_distilbert_kan_7["test_metrics"].overall_stat["Kappa"],
        ],
        [   9,
            imdb_distilbert_kan_9["test_metrics"].overall_stat["Overall ACC"],
            imdb_distilbert_kan_9["test_metrics"].overall_stat["F1 Macro"],
            imdb_distilbert_kan_9["test_metrics"].overall_stat["F1 Micro"],
            imdb_distilbert_kan_9["test_metrics"].overall_stat["Kappa"],
        ],
    ],
    columns=["grid_size", "Overall ACC", "F1 Macro", "F1 Micro", "Kappa"],
)

In [7]:
import pickle
import pandas as pd

with open("results/ablation_agnews_distilbert_40_30924.pickle","rb",) as f:
    agnews_distilbert_40 = pickle.load(f)

with open("results/ablation_dbpedia_distilbert_138_108068.pickle", "rb") as f:
    dbpedia_distilbert_138 = pickle.load(f)

with open("results/ablation_imdb_distilbert_20_15422.pickle","rb",) as f:
    imdb_distilbert_20 = pickle.load(f)

with open("results/ablation_papluca_distilbert_196_154664.pickle", "rb") as f:
    papluca_distilbert_196 = pickle.load(f)

with open("results/ablation_sst5_distilbert_50_38705.pickle","rb",) as f:
    sst5_distilbert_50 = pickle.load(f)

with open("results/ablation_trec_distilbert_477_512825.pickle", "rb") as f:
    trec_distilbert_477 = pickle.load(f)

with open("results/ablation_yelp_distilbert_50_38705.pickle", "rb") as f:
    yelp_distilbert_50 = pickle.load(f)


pd.DataFrame(
    [
        [
            agnews_distilbert_40["test_metrics"].overall_stat["Overall ACC"],
            agnews_distilbert_40["test_metrics"].overall_stat["F1 Macro"],
            agnews_distilbert_40["test_metrics"].overall_stat["F1 Micro"],
            agnews_distilbert_40["test_metrics"].overall_stat["Kappa"],
        ],
        [
            dbpedia_distilbert_138["test_metrics"].overall_stat["Overall ACC"],
            dbpedia_distilbert_138["test_metrics"].overall_stat["F1 Macro"],
            dbpedia_distilbert_138["test_metrics"].overall_stat["F1 Micro"],
            dbpedia_distilbert_138["test_metrics"].overall_stat["Kappa"],
        ],
        [
            imdb_distilbert_20["test_metrics"].overall_stat["Overall ACC"],
            imdb_distilbert_20["test_metrics"].overall_stat["F1 Macro"],
            imdb_distilbert_20["test_metrics"].overall_stat["F1 Micro"],
            imdb_distilbert_20["test_metrics"].overall_stat["Kappa"],
        ],
        [
            papluca_distilbert_196["test_metrics"].overall_stat["Overall ACC"],
            papluca_distilbert_196["test_metrics"].overall_stat["F1 Macro"],
            papluca_distilbert_196["test_metrics"].overall_stat["F1 Micro"],
            papluca_distilbert_196["test_metrics"].overall_stat["Kappa"],
        ],
        [
            sst5_distilbert_50["test_metrics"].overall_stat["Overall ACC"],
            sst5_distilbert_50["test_metrics"].overall_stat["F1 Macro"],
            sst5_distilbert_50["test_metrics"].overall_stat["F1 Micro"],
            sst5_distilbert_50["test_metrics"].overall_stat["Kappa"],
        ],
        [
            trec_distilbert_477["test_metrics"].overall_stat["Overall ACC"],
            trec_distilbert_477["test_metrics"].overall_stat["F1 Macro"],
            trec_distilbert_477["test_metrics"].overall_stat["F1 Micro"],
            trec_distilbert_477["test_metrics"].overall_stat["Kappa"],
        ],
        [
            yelp_distilbert_50["test_metrics"].overall_stat["Overall ACC"],
            yelp_distilbert_50["test_metrics"].overall_stat["F1 Macro"],
            yelp_distilbert_50["test_metrics"].overall_stat["F1 Micro"],
            yelp_distilbert_50["test_metrics"].overall_stat["Kappa"],
        ],
    ],
    columns=["Overall ACC", "F1 Macro", "F1 Micro", "Kappa"],
)

,Overall ACC,F1 Macro,F1 Micro,Kappa
0,0.834667,0.830679,0.834667,0.779213
1,0.892444,0.891260,0.892444,0.884062
2,0.777778,0.776677,0.777778,0.556271
3,0.816000,0.819351,0.816000,0.806097
4,0.351321,0.176076,0.351321,0.116727
5,0.179171,0.006466,0.179171,0.000000
6,0.458222,0.455761,0.458222,0.322462


In [8]:
with open("results/sst5_distilbert_kan.pickle","rb",) as f:
    sst5_distilbert_kan = pickle.load(f)

with open("results/trec_distilbert_kan.pickle", "rb") as f:
    trec_distilbert_kan = pickle.load(f)


pd.DataFrame(
    [
        [
            sst5_distilbert_kan["test_metrics"].overall_stat["Overall ACC"],
            sst5_distilbert_kan["test_metrics"].overall_stat["F1 Macro"],
            sst5_distilbert_kan["test_metrics"].overall_stat["F1 Micro"],
            sst5_distilbert_kan["test_metrics"].overall_stat["Kappa"],
        ],
        [
            trec_distilbert_kan["test_metrics"].overall_stat["Overall ACC"],
            trec_distilbert_kan["test_metrics"].overall_stat["F1 Macro"],
            trec_distilbert_kan["test_metrics"].overall_stat["F1 Micro"],
            trec_distilbert_kan["test_metrics"].overall_stat["Kappa"],
        ]
    ],
    columns=["Overall ACC", "F1 Macro", "F1 Micro", "Kappa"],
)

,Overall ACC,F1 Macro,F1 Micro,Kappa
0,0.400787,0.335580,0.400787,0.207863
1,0.350504,0.057369,0.350504,0.257868
